In [3]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

/home/work/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-02-07 04:50:29.408066: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 04:50:29.647607: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-07 04:50:33.464948: W tensorflow/compiler/xla/stream_executor/platform/default/

In [5]:
kospi_train = fdr.DataReader('KS11', '2015-01-01','2021-03-31')
kospi_train['Change1'] = kospi_train['Close']/kospi_train['Close'].shift(1) - 1

kospi_train = kospi_train.dropna()
print(kospi_train.head())
print(kospi_train.columns)
print(kospi_train.shape)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2015-01-05  1921.959961  1921.959961  1903.369995  1915.750000  1915.750000   
2015-01-06  1895.479980  1900.020020  1877.380005  1882.449951  1882.449951   
2015-01-07  1878.900024  1887.599976  1876.270020  1883.829956  1883.829956   
2015-01-08  1895.849976  1907.229980  1891.770020  1904.650024  1904.650024   
2015-01-09  1919.800049  1929.099976  1915.849976  1924.699951  1924.699951   

              Volume   Change1  
Date                            
2015-01-05  313400.0 -0.005549  
2015-01-06  303700.0 -0.017382  
2015-01-07  280500.0  0.000733  
2015-01-08  262200.0  0.011052  
2015-01-09  308800.0  0.010527  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')
(1531, 7)


In [6]:
kospi_test = fdr.DataReader('KS11', '2021-04-01','2022-10-31')
kospi_test['Change1'] = kospi_test['Close']/kospi_test['Close'].shift(1) - 1

kospi_test = kospi_test.dropna()
print(kospi_test.head())
print(kospi_test.columns)
print(kospi_test.shape)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2021-04-02  3104.719971  3121.580078  3101.600098  3112.800049  3112.800049   
2021-04-05  3121.229980  3127.199951  3101.860107  3120.830078  3120.830078   
2021-04-06  3123.800049  3139.290039  3110.189941  3127.080078  3127.080078   
2021-04-07  3129.070068  3146.189941  3125.330078  3137.409912  3137.409912   
2021-04-08  3137.250000  3145.810059  3122.090088  3143.260010  3143.260010   

             Volume   Change1  
Date                           
2021-04-02   792700  0.008227  
2021-04-05  1084800  0.002580  
2021-04-06  1279000  0.002003  
2021-04-07  1984800  0.003303  
2021-04-08  1357300  0.001865  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')
(388, 7)


In [7]:
kospi_train.index = kospi_train.index.strftime('%Y-%m-%d')
print(kospi_train.index)

kospi_test.index = kospi_test.index.strftime('%Y-%m-%d')
print(kospi_test.index)

Index(['2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09',
       '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16',
       ...
       '2021-03-18', '2021-03-19', '2021-03-22', '2021-03-23', '2021-03-24',
       '2021-03-25', '2021-03-26', '2021-03-29', '2021-03-30', '2021-03-31'],
      dtype='object', name='Date', length=1531)
Index(['2021-04-02', '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08',
       '2021-04-09', '2021-04-12', '2021-04-13', '2021-04-14', '2021-04-15',
       ...
       '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21', '2022-10-24',
       '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', '2022-10-31'],
      dtype='object', name='Date', length=388)


In [8]:
import json
embedding = open('/home/work/nlp/embedding/result/news_title_128.json', encoding='utf-8')
em_dict = json.load(embedding)

In [9]:
ke_vec_train = []
ke_vec_test = []

for index, row in kospi_train.iterrows():
    a = em_dict[index].copy()
    a.insert(0, row['Change1'])
    ke_vec_train.append(a)

for index, row in kospi_test.iterrows():
    a = em_dict[index].copy()
    a.insert(0, row['Change1'])
    ke_vec_test.append(a)

In [10]:
kospi_train_c1 = kospi_train['Change1']
kospi_train_c1 = 100 * kospi_train_c1.values

kospi_test_c1 = kospi_test['Change1']
kospi_test_c1 = 100 * kospi_test_c1.values

In [11]:
lookback_window = 60
x_train_1, y_train_1 = [], []
x_test_1, y_test_1 = [], []

In [12]:
for i in range(lookback_window, len(ke_vec_train)-1):
    x_train_1.append(ke_vec_train[i - lookback_window:i])
    y_train_1.append(kospi_train_c1[i])
    
for index in range(len(y_train_1)):
    if y_train_1[index]>= -0.1 and y_train_1[index] < 0.1:
        y_train_1[index] = 'neutral'
    elif y_train_1[index] >= 0.1:
        y_train_1[index] = 'increase'
    else:
        y_train_1[index] = 'decrease' 

In [13]:
x_train_1 = np.array(x_train_1)
y_train_1 = np.array(y_train_1)
print(x_train_1.shape, y_train_1.shape)

(1470, 60, 129) (1470,)


In [14]:
for i in range(lookback_window, len(ke_vec_test)-1):
    x_test_1.append(ke_vec_test[i - lookback_window:i])
    y_test_1.append(kospi_test_c1[i])
    
for index in range(len(y_test_1)):
    if y_test_1[index]>= -0.1 and y_test_1[index] < 0.1:
        y_test_1[index] = 'neutral'
    elif y_test_1[index] >= 0.1:
        y_test_1[index] = 'increase'
    else:
        y_test_1[index] = 'decrease'    

In [15]:
x_test_1 = np.array(x_test_1)
y_test_1 = np.array(y_test_1)
print(x_test_1.shape, y_test_1.shape)

(327, 60, 129) (327,)


In [16]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_1)

y_train_1_e = to_categorical(encoder.transform(y_train_1))
y_test_1_e = to_categorical(encoder.transform(y_test_1))

print(y_train_1_e.shape, y_test_1_e.shape)

(1470, 3) (327, 3)


In [17]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=2,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
        ),
    Dense(3, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
###################################################################
model_2 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=3,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
        ),
    Dense(3, activation='softmax')
])

model_2.summary()
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
###################################################################
model_3 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=4,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
        ),
    Dense(3, activation='softmax')
])

model_3.summary()
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-02-07 04:50:41.229197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 04:50:41.914668: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10315 MB memory:  -> device: 0, name: CUDA GPU, pci bus id: 0000:d5:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 100)               461000    
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                                 
Total params: 461,303
Trainable params: 461,303
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_1 (TCN)                 (None, 100)               683900    
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 684,203
Traina

In [ ]:
print('Train model_1...')
model_1.fit(x_train_1, y_train_1_e, epochs=30, verbose=1)
model_1.save('/home/work/nlp/TCN_models/classification_model/embedding/1day/kospi_embedding_1day_classification_k2.h5')

print('Train model_2...')
model_2.fit(x_train_1, y_train_1_e, epochs=30, verbose=1)
model_2.save('/home/work/nlp/TCN_models/classification_model/embedding/1day/kospi_embedding_1day_classification_k3.h5')

print('Train model_3...')
model_3.fit(x_train_1, y_train_1_e, epochs=30, verbose=1)
model_3.save('/home/work/nlp/TCN_models/classification_model/embedding/1day/kospi_embedding_1day_classification_k4.h5')

Train model_1...
Epoch 1/30
46/46 [==============================] - 73s 953ms/step - loss: 1.4999 - accuracy: 0.4136
Epoch 2/30
46/46 [==============================] - 3s 60ms/step - loss: 0.9372 - accuracy: 0.5286
Epoch 3/30
46/46 [==============================] - 1s 31ms/step - loss: 0.8459 - accuracy: 0.6095
Epoch 4/30
46/46 [==============================] - 1s 32ms/step - loss: 0.6827 - accuracy: 0.7231
Epoch 5/30
46/46 [==============================] - 1s 29ms/step - loss: 0.4925 - accuracy: 0.8054
Epoch 6/30
46/46 [==============================] - 1s 29ms/step - loss: 0.3173 - accuracy: 0.8844
Epoch 7/30
46/46 [==============================] - 1s 30ms/step - loss: 0.1948 - accuracy: 0.9313
Epoch 8/30
46/46 [==============================] - 1s 30ms/step - loss: 0.2094 - accuracy: 0.9156
Epoch 9/30
46/46 [==============================] - 1s 28ms/step - loss: 0.1435 - accuracy: 0.9476
Epoch 10/30
46/46 [==============================] - 1s 31ms/step - loss: 0.0630 - accurac

In [ ]:
model_1.evaluate(x_test_1, y_test_1_e)[1]
model_2.evaluate(x_test_1, y_test_1_e)[1]
model_3.evaluate(x_test_1, y_test_1_e)[1]